In [ ]:
# hddm
import hddm
import kabuki
import pymc

# design matrix
from patsy import dmatrix

# data analysis
import pandas as pd
import numpy as np

# plot
import seaborn as sns
import matplotlib.pyplot as plt

# parallel computation
from joblib import Parallel, delayed

# rmANOVA
from statsmodels.stats.anova import AnovaRM

import os,sys

In [ ]:
print('The current HDDM version is', hddm.__version__) # 0.8.0
print('The current Kabuki version is', kabuki.__version__) # 0.6.3
print('The current PyMC version is', pymc.__version__) # 2.3.8
print('The current Numpy version is', np.__version__) 
print('The current Pandas version is', pd.__version__)
print('The current seaborn version is', sns.__version__)

# Experiment Design

It is a car-face discrimination task in a 2* 2  factorial within-subject design. 

Experimental factors **"stimulus coherence”(with level "low" and "high")** and **"spatial prioritization"(with level "yes" and "no")**.

Experimental manipulation has four levels:




|level|coherence|prioritization|
| -------- | -------- | -------- |
|1|high|yes|
|2|high|no|
|3|low|yes|
|4|low|no|


![Image Name](https://cdn.kesci.com/upload/image/rmm71hlded.png?imageView2/0/w/960/h/960)


we assume that the effect of experimental design is reflected in underlying latent variable. 

Different levels corresponds to different parameters.

For the moment, we assume **phase coherence** can influence **drift rate(v)** and **spatial priotiziation** can influence **initial bias(z)**.

Because phase coherence have two levels(high & low) and spatial priotization(yes & no) have two levels, we have four levels together.

# Situation1

## Simulate data for the experiment

Because we need to code the response according to the stimulus, so we must transform the varibale 'stimulus'.

We assume that stimulus influence the parameter 'z'.

Due to the response is pressing 'car' or 'face', instead of accuracy coding, we set the stimulus(button press) as the boundaries.

So, when there are some bias, one stimulus is ``z``, another is ``1-z``.

In [ ]:
# set the random seed
np.random.seed(123)

# set number of subjects and number of trials per level for the simulated experiment.
n_subjects = 20
trials_per_level = 72

# set up parameters of DDM for four levels of the stimulus face(1).
level1_a = {'v':.8, 'a':2, 't':.3, 'z':.8, 'sv':0, 'sz':0, 'st':0}
level2_a = {'v':.8, 'a':2, 't':.3, 'z':.5, 'sv':0, 'sz':0, 'st':0}
level3_a = {'v':.2, 'a':2, 't':.3, 'z':.8, 'sv':0, 'sz':0, 'st':0}
level4_a = {'v':.2, 'a':2, 't':.3, 'z':.5, 'sv':0, 'sz':0, 'st':0}

# set up parameters of DDM for four levels of the stimulus car(0).
level1_b = {'v':.8, 'a':2, 't':.3, 'z':.2, 'sv':0, 'sz':0, 'st':0}
level2_b = {'v':.8, 'a':2, 't':.3, 'z':.5, 'sv':0, 'sz':0, 'st':0}
level3_b = {'v':.2, 'a':2, 't':.3, 'z':.2, 'sv':0, 'sz':0, 'st':0}
level4_b = {'v':.2, 'a':2, 't':.3, 'z':.5, 'sv':0, 'sz':0, 'st':0}

# generate simulated data
np.random.seed(123)
data_a, params_a = hddm.generate.gen_rand_data({'level1':level1_a,
                                             'level2':level2_a,
                                             'level3':level3_a,
                                             'level4':level4_a},
                                             size = trials_per_level,
                                             subjs = n_subjects)
data_b, params_b = hddm.generate.gen_rand_data({'level1':level1_b,
                                             'level2':level2_b,
                                             'level3':level3_b,
                                             'level4':level4_b},
                                             size = trials_per_level,
                                             subjs = n_subjects
)
# add column stimulus
data_a['stimulus'] = pd.Series(np.ones((len(data_a))), index=data_a.index)   # face
data_b['stimulus'] = pd.Series(np.ones((len(data_b)))*0, index=data_a.index) # car

# combine the data from two stimulus
data = data_a.append(data_b,ignore_index=True)

# add column coherence
data.loc[(data['condition']=='level1')|(data['condition']=='level2'),'coherence'] = 1    
data.loc[(data['condition']=='level3')|(data['condition']=='level4'),'coherence'] = 0

# add column spatial priotiziation
data.loc[(data['condition']=='level1')|(data['condition']=='level3'),'attention'] = 1    
data.loc[(data['condition']=='level2')|(data['condition']=='level4'),'attention'] = 0

## check the data

In [ ]:
data

we can check how many trials per condition.

In [ ]:
data.groupby(['subj_idx','stimulus','condition','coherence','attention']).size()

In [ ]:
data.groupby(['stimulus','coherence','attention','response'])['rt'].agg(['size','mean','std']).head(20)

In [ ]:
data.groupby(['response'])['rt'].agg(['size','mean','std'])

In [ ]:
data.groupby(['stimulus','response'])['rt'].agg(['size','mean','std'])

In [ ]:
data.groupby(['coherence','response'])['rt'].agg(['size','mean','std'])

In [ ]:
data.groupby(['attention','response'])['rt'].agg(['size','mean','std'])

we can check the effect of the coherence

In [ ]:
data.groupby(['stimulus','coherence','response'])['rt'].agg(['size','mean','std'])

Also, we can check the effect of the attention

In [ ]:
data.groupby(['stimulus','attention','response'])['rt'].agg(['size','mean','std'])

## anova

In [ ]:
# ANOVA
model_aovrm3way = AnovaRM(data,
                   'rt',
                   'subj_idx',
                   within=['coherence','attention','stimulus'],
                   aggregate_func='mean')
res3way=model_aovrm3way.fit()
print(res3way)


In [ ]:
# ANOVA
model_aovrm3way = AnovaRM(data,
                   'response',
                   'subj_idx',
                   within=['coherence','attention','stimulus'],
                   aggregate_func='mean')
res3way=model_aovrm3way.fit()
print(res3way)

The parameter **v** for high coherence is 0.8;

the parameter **v** for low coherence is 0.2;

the parameter **z** for cue of stimulus 1 is 0.8 and of stimulus 0 is 0.2;

the parameter **z** for no cue is 0.5;

At the same time, we assume that the response is mapping into two stimulus, car or face.

### model1 

v~coherence,

z~prioritization

when we fit the data, we will face the ideal situation.

$v=\alpha_0 + \alpha_1*C(coherence)$

$v_{low}=\alpha_0 = 0.2$

$v_{high}=\alpha_0 + \alpha_1 = 0.8$


$z=\beta_0 + \beta_1*C(prioritization)$ + z_link_func

stim = 1

$z_{no}=1 - \beta_0 = 0.5$

$z_{yes}=1 - (\beta_0 + \beta_1) = 0.2$

stim = 0

$z_{no}=\beta_0 = 0.5$

$z_{yes}=\beta_0 + \beta_1 = 0.8$

We expect the parameter estimated is:

$\alpha_0 = 0.2, \alpha_1 = 0.6$

$\beta_0 = 0.5, \beta_1 = 0.3$

### model2

if we set $v~C(coherence)*C(prioritization), z~C(coherence)*C(prioritization)$.

$v=\alpha_0 + \alpha_1*C(coherence) + \alpha_2*C(prioritization) + \alpha_3*C(coherence)*C(priorization)$

$v_{low,yes}=\alpha_0 + \alpha_2 + = 0.2$

$v_{high,yes}=\alpha_0 + \alpha_1 + \alpha_2 + \alpha_3= 0.8$

$v_{low,no}=\alpha_0  = 0.2$

$v_{high,no}=\alpha_0 + \alpha_1 = 0.8$

$v=\beta_0 + \beta_1*C(coherence) + \beta_2*C(prioritization) + \beta_3*C(coherence)*C(priorization)$ + z_link_func

stim = 1

$z_{no,high}=1 - \beta_0 - \beta_1= 0.5$

$z_{yes,high}=1 - (\beta_0 + \beta_1 + \beta_2 +\beta_3) = 0.2$

$z_{no,low}=1 - \beta_0 = 0.5$

$z_{yes,low}=1 - (\beta_0 + \beta_2) = 0.2$

stim = 0

$z_{no,high}=\beta_0 + \beta_1 = 0.5$

$z_{yes,high}=\beta_0 + \beta_1 + \beta_2 + \beta_3 = 0.8$

$z_{no,low}=\beta_0 = 0.5$

$z_{yes,low}=\beta_0 + \beta_2 = 0.8$

We expect the parameter estimated is:

$\alpha_0 = 0.2, \alpha_1 = 0.6, \alpha_2 = 0, \alpha_3 = 0$

$\beta_0 = 0.5, \beta_2 = 0.3, \beta_1 = 0, \beta_3 = 0$

this is the link func. the function is to create the node of z,

for stimulus 0, the node of z is $\beta_0 + \beta_1*X$

for stimulus 1, the node of z is $1 - (\beta_0 + \beta_1*X)$

In [ ]:
def z_link_func(x, data=data):
    stim = (np.asarray(dmatrix('0 + C(s, [[0], [1]])',
                              {'s': data.stimulus.loc[x.index]},return_type='dataframe'))
    )
    # Apply z = (1 - x) to flip them along 0.5
    z_flip = np.subtract(stim, x.to_frame())
    # The above inverts those values we do not want to flip,
    # so invert them back
    z_flip[stim == 0] *= -1
    return z_flip

## Fit data

### model1 

In [ ]:
v_reg = {'model':'v~C(coherence)','link_func':lambda x:x}
z_reg = {'model':'z~C(attention)','link_func':z_link_func}
reg_descr = [v_reg, z_reg]
m_reg = hddm.HDDMRegressor(data, reg_descr, include='z')

because we construct a glm, so we can check the design matrix and parameters.

this is the parameters.

In [ ]:
# hddm/models/hddm_regression.py/line 164
params=np.matrix(m_reg.model_descrs[1]['params'])
params

this is the design matrixs.

In [ ]:
# hddm/models/hddm_regression.py/line 154
design_matrix=dmatrix('C(attention)',
                data,
                return_type="dataframe",
                NA_action="raise")
design_matrix

` hddm/models/hddm_regression.py/line 174`

`design_matrix.dot(params.T)`

the row and line is nx2 2x1

the result is $\beta_0 + \beta_1 * C(attention)$

Then the linear predictor need to be transformed via link function

```
def z_link_func(x, data=data):
    stim = (np.asarray(dmatrix('0 + C(s, [[0], [1]])',
                              {'s': data.stimulus.loc[x.index]},return_type='dataframe'))
    )
    # Apply z = (1 - x) to flip them along 0.5
    z_flip = np.subtract(stim, x.to_frame())
    # The above inverts those values we do not want to flip,
    # so invert them back
    z_flip[stim == 0] *= -1
    return z_flip
```

In [ ]:
np.asarray(dmatrix('0 + C(s, [[0], [1]])',
                              {'s': data.stimulus.loc[data.index]},return_type='dataframe'))

for stimulus 0, the result is $\beta_0 + \beta_1*X$

for stimulus 1, the result is $1 - (\beta_0 + \beta_1*X)$

In [ ]:
v_reg = {'model':'v~C(coherence)','link_func':lambda x:x}
z_reg = {'model':'z~C(attention)','link_func':lambda x:x}
reg_descr = [v_reg, z_reg]
m_reg = hddm.HDDMRegressor(data, reg_descr, include='z')
m_reg.find_starting_values()
m_reg.sample(2000,burn=1000)

In [ ]:
m_reg.plot_posteriors()

### model2

the glm and link function is similiar to model1.

In [ ]:
v_reg = {'model':'v~C(coherence)*C(attention)','link_func':lambda x:x}
z_reg = {'model':'z~C(coherence)*C(attention)','link_func':z_link_func}
reg_descr = [v_reg, z_reg]
m_reg2 = hddm.HDDMRegressor(data, reg_descr, include='z')
m_reg2.find_starting_values()
m_reg2.sample(2000,burn=1000)

In [ ]:
m_reg2.plot_posteriors()

### model3

In model3, we want to test if we use incorrect coding, the results of the model.

In [ ]:
data_tmp = data
data_tmp.loc[(data_tmp['response']==1)&(data_tmp['stimulus']==0),'_response'] = 0 
data_tmp.loc[(data_tmp['response']==1)&(data_tmp['stimulus']==1),'_response'] = 1 
data_tmp.loc[(data_tmp['response']==0)&(data_tmp['stimulus']==0),'_response'] = 1
data_tmp.loc[(data_tmp['response']==0)&(data_tmp['stimulus']==1),'_response'] = 0
data_tmp['response'] = data_tmp['_response']

In [ ]:
data_tmp

we can check how many trials per condition.

In [ ]:
data_tmp.groupby(['subj_idx','stimulus','condition','coherence','attention']).size()

In [ ]:
data_tmp.groupby(['stimulus','coherence','attention','response'])['rt'].agg(['size','mean','std']).head(20)

In [ ]:
data_tmp.groupby(['response'])['rt'].agg(['size','mean','std'])

In [ ]:
data_tmp.groupby(['stimulus','response'])['rt'].agg(['size','mean','std'])

we can check the effect of the coherence

In [ ]:
data_tmp.groupby(['stimulus','coherence','response'])['rt'].agg(['size','mean','std'])

Also, we can check the effect of the attention

In [ ]:
data_tmp.groupby(['stimulus','attention','response'])['rt'].agg(['size','mean','std'])

In [ ]:
v_reg = {'model':'v~C(coherence)','link_func':lambda x:x}
z_reg = {'model':'z~C(attention)','link_func':z_link_func}
reg_descr = [v_reg, z_reg]
m_reg3 = hddm.HDDMRegressor(data_tmp, reg_descr, include='z')
m_reg3.find_starting_values()
m_reg3.sample(2000,burn=1000)

In [ ]:
m_reg3.plot_posteriors()

# Simulation2

## simulate the data

To advance the simulated data similar to our true data, we adjust the value of parameter, and construct the hierarchical structure of every subjects.

In this part, we assume that the prioritization affect the non-decision time

In [515]:
# set number of subjects and number of trials per level for the simulated experiment.
n_subjects = 15
trials_per_level = 36

# set the group parameter
# intercept and slope of v
v_int = 2
v_coh = 1.2
v_sig1 = 1
v_sig2 = 1
# intercept and slope of t
t_int = 0.3
t_pri = 0.1
t_sig1 = 0.1
t_sig2 = 0.1
# intercept of a
a_int = 1.4
a_sig = 0.2
# intercept of z
z_int = 0.5
z_sig = 0.1

# set the subject parameter
# v
v_int_subj = np.random.normal(v_int,v_sig1,[n_subjects,2])
v_coh_subj = np.random.normal(v_coh,v_sig2,[n_subjects,2])
# t
t_int_subj = np.random.normal(t_int,t_sig1,[n_subjects,2])
t_pri_subj = np.random.normal(t_pri,t_sig2,[n_subjects,2])
# a
a_int_subj = np.random.normal(a_int,a_sig,[n_subjects,2])
# z
z_int_subj = np.random.normal(z_int,z_sig,[n_subjects,2])

# dataframe
df = pd.DataFrame()
for i in range(n_subjects):
    # the different v of two experimental factor 
    v_high  = v_int_subj[i] + v_coh_subj[i]
    v_low = v_int_subj[i]
    # the different t of two experimental factor
    t_yes = t_int_subj[i] + t_pri_subj[i]
    t_no = t_int_subj[i]
    # the default parameter a
    a = a_int_subj[i]
    # the default parameter z
    z = z_int_subj[i]
    # set up parameters of DDM for four levels of the stimulus face(1).
    level1_a = {'v':v_high[0], 'a':a[0], 't':t_yes[0], 'z':z[0], 'sv':0, 'sz':0, 'st':0}
    level2_a = {'v':v_high[0], 'a':a[0], 't':t_no[0], 'z':z[0], 'sv':0, 'sz':0, 'st':0}
    level3_a = {'v':v_low[0], 'a':a[0], 't':t_yes[0], 'z':z[0], 'sv':0, 'sz':0, 'st':0}
    level4_a = {'v':v_low[0], 'a':a[0], 't':t_no[0], 'z':z[0], 'sv':0, 'sz':0, 'st':0}

    # set up parameters of DDM for four levels of the stimulus car(0).
    level1_b = {'v':v_high[1], 'a':a[1], 't':t_yes[1], 'z':z[1], 'sv':0, 'sz':0, 'st':0}
    level2_b = {'v':v_high[1], 'a':a[1], 't':t_no[1], 'z':z[1], 'sv':0, 'sz':0, 'st':0}
    level3_b = {'v':v_low[1], 'a':a[1], 't':t_yes[1], 'z':z[1], 'sv':0, 'sz':0, 'st':0}
    level4_b = {'v':v_low[1], 'a':a[1], 't':t_no[1], 'z':z[1], 'sv':0, 'sz':0, 'st':0}

    # generate simulated data
    np.random.seed(123)
    data_a, params_a = hddm.generate.gen_rand_data({'level1':level1_a,
                                                 'level2':level2_a,
                                                 'level3':level3_a,
                                                 'level4':level4_a},
                                                 size = trials_per_level
                                                  )
    data_b, params_b = hddm.generate.gen_rand_data({'level1':level1_b,
                                                 'level2':level2_b,
                                                 'level3':level3_b,
                                                 'level4':level4_b},
                                                 size = trials_per_level
                                                 )
    # add column stimulus
    data_a['stimulus'] = pd.Series(np.ones((len(data_a))), index=data_a.index)   # face
    data_b['stimulus'] = pd.Series(np.ones((len(data_b)))*0, index=data_a.index) # car

    # combine the data from two stimulus
    data = data_a.append(data_b,ignore_index=True)
    
    # add subject
    data['subj_idx'] = pd.Series(np.ones((len(data)))*i, index=data.index) 

    # add column coherence
    data.loc[(data['condition']=='level1')|(data['condition']=='level2'),'coherence'] = 1    
    data.loc[(data['condition']=='level3')|(data['condition']=='level4'),'coherence'] = 0

    # add column spatial priotiziation
    data.loc[(data['condition']=='level1')|(data['condition']=='level3'),'attention'] = 1    
    data.loc[(data['condition']=='level2')|(data['condition']=='level4'),'attention'] = 0
    
    df = df.append(data)

In [ ]:
# the stimulus coding
df.loc[(df['stimulus']==0)&(df['response']==0),'_response']=1
df.loc[(df['stimulus']==1)&(df['response']==1),'_response']=1
df.loc[(df['stimulus']==0)&(df['response']==1),'_response']=0
df.loc[(df['stimulus']==1)&(df['response']==0),'_response']=0

In [ ]:
df

## check the data

In [ ]:
df.groupby(['response'])['rt'].agg(['size','mean','std'])

In [ ]:
df.groupby(['_response'])['rt'].agg(['size','mean','std'])

In [ ]:
df.groupby(['stimulus','response'])['rt'].agg(['size','mean','std'])

In [ ]:
df.groupby(['coherence','response'])['rt'].agg(['size','mean','std'])

In [ ]:
df.groupby(['attention','response'])['rt'].agg(['size','mean','std'])

In [ ]:
df.groupby(['stimulus','coherence','response'])['rt'].agg(['size','mean','std'])

In [ ]:
df.groupby(['stimulus','attention','response'])['rt'].agg(['size','mean','std'])

In [ ]:
df.groupby(['stimulus','coherence','attention','response'])['rt'].agg(['size','mean','std'])

## ANOVA 

In [516]:
# ANOVA for rt
model_aovrm3way = AnovaRM(df,
                   'rt',
                   'subj_idx',
                   within=['coherence','attention','stimulus'],
                   aggregate_func='mean')
res3way=model_aovrm3way.fit()
print(res3way)
# ANOVA for response
model_aovrm3way = AnovaRM(df,
                   'response',
                   'subj_idx',
                   within=['coherence','attention','stimulus'],
                   aggregate_func='mean')
res3way=model_aovrm3way.fit()
print(res3way)

                          Anova
                             F Value Num DF  Den DF Pr > F
----------------------------------------------------------
coherence                    53.9040 1.0000 14.0000 0.0000
attention                    21.3669 1.0000 14.0000 0.0004
stimulus                      1.4885 1.0000 14.0000 0.2426
coherence:attention          24.4169 1.0000 14.0000 0.0002
coherence:stimulus            3.9825 1.0000 14.0000 0.0658
attention:stimulus            8.5862 1.0000 14.0000 0.0110
coherence:attention:stimulus 49.8911 1.0000 14.0000 0.0000

                          Anova
                             F Value Num DF  Den DF Pr > F
----------------------------------------------------------
coherence                    38.8132 1.0000 14.0000 0.0000
attention                    25.4783 1.0000 14.0000 0.0002
stimulus                      3.7692 1.0000 14.0000 0.0726
coherence:attention           0.9742 1.0000 14.0000 0.3404
coherence:stimulus            4.5829 1.0000 14.000

## fit data

# Simulation3

## simulate the data

In [513]:
# set number of subjects and number of trials per level for the simulated experiment.
n_subjects = 15
trials_per_level = 36

# set the group parameter
# intercept and slope of v
v_int = 1.7
v_coh = 1.3
v_sig1 = 0.7
v_sig2 = 1
# intercept and slope of z
z_int = 0.5
z_pri = 0.1
z_sig1 = 0.1
z_sig2 = 0.1
# intercept of a
a_int = 1.4
a_sig = 0.2
# intercept of t
t_int = 0.36
t_sig = 0.02

# set the subject parameter
# v
v_int_subj = np.random.normal(v_int,v_sig1,[n_subjects,2])
v_coh_subj = np.random.normal(v_coh,v_sig2,[n_subjects,2])
# z
z_int_subj = np.random.normal(z_int,z_sig1,[n_subjects,2])
z_pri_subj = np.random.normal(z_pri,z_sig2,[n_subjects,2])
# a
a_int_subj = np.random.normal(a_int,a_sig,[n_subjects,2])
# t
t_int_subj = np.random.normal(t_int,t_sig,[n_subjects,2])

# dataframe
df = pd.DataFrame()
for i in range(n_subjects):
    
    # the different v of two experimental factor 
    v_high  = v_int_subj[i] + v_coh_subj[i]
    v_low = v_int_subj[i]
    # the different t of two experimental factor
    z_yes = z_int_subj[i] + z_pri_subj[i]
    z_no = z_int_subj[i]
    # the default parameter a
    a = a_int_subj[i]
    # the default parameter t
    t = t_int_subj[i]
    
    # set up parameters of DDM for four levels of the stimulus face(1).
    level1_a = {'v':v_high[0], 'a':a[0], 't':t[0], 'z':z_yes[0], 'sv':0, 'sz':0, 'st':0}
    level2_a = {'v':v_high[0], 'a':a[0], 't':t[0], 'z':z_no[0], 'sv':0, 'sz':0, 'st':0}
    level3_a = {'v':v_low[0], 'a':a[0], 't':t[0], 'z':z_yes[0], 'sv':0, 'sz':0, 'st':0}
    level4_a = {'v':v_low[0], 'a':a[0], 't':t[0], 'z':z_no[0], 'sv':0, 'sz':0, 'st':0}

    # set up parameters of DDM for four levels of the stimulus car(0).
    level1_b = {'v':v_high[1], 'a':a[1], 't':t[1], 'z':z_yes[0], 'sv':0, 'sz':0, 'st':0}
    level2_b = {'v':v_high[1], 'a':a[1], 't':t[1], 'z':z_no[0], 'sv':0, 'sz':0, 'st':0}
    level3_b = {'v':v_low[1], 'a':a[1], 't':t[1], 'z':z_yes[0], 'sv':0, 'sz':0, 'st':0}
    level4_b = {'v':v_low[1], 'a':a[1], 't':t[1], 'z':z_no[0], 'sv':0, 'sz':0, 'st':0}

    # generate simulated data
    np.random.seed(123)
    data_a, params_a = hddm.generate.gen_rand_data({'level1':level1_a,
                                                 'level2':level2_a,
                                                 'level3':level3_a,
                                                 'level4':level4_a},
                                                 size = trials_per_level
                                                  )
    data_b, params_b = hddm.generate.gen_rand_data({'level1':level1_b,
                                                 'level2':level2_b,
                                                 'level3':level3_b,
                                                 'level4':level4_b},
                                                 size = trials_per_level
                                                 )
    # add column stimulus
    data_a['stimulus'] = pd.Series(np.ones((len(data_a))), index=data_a.index)   # face
    data_b['stimulus'] = pd.Series(np.ones((len(data_b)))*0, index=data_a.index) # car

    # combine the data from two stimulus
    data = data_a.append(data_b,ignore_index=True)
    
    # add subject
    data['subj_idx'] = pd.Series(np.ones((len(data)))*i, index=data.index) 

    # add column coherence
    data.loc[(data['condition']=='level1')|(data['condition']=='level2'),'coherence'] = 1    
    data.loc[(data['condition']=='level3')|(data['condition']=='level4'),'coherence'] = 0

    # add column spatial priotiziation
    data.loc[(data['condition']=='level1')|(data['condition']=='level3'),'attention'] = 1    
    data.loc[(data['condition']=='level2')|(data['condition']=='level4'),'attention'] = 0
    
    df = df.append(data)

In [481]:
# the stimulus coding
df.loc[(df['stimulus']==0)&(df['response']==0),'_response']=1
df.loc[(df['stimulus']==1)&(df['response']==1),'_response']=1
df.loc[(df['stimulus']==0)&(df['response']==1),'_response']=0
df.loc[(df['stimulus']==1)&(df['response']==0),'_response']=0

## check the data

In [ ]:
df

In [ ]:
df.groupby(['response'])['rt'].agg(['size','mean','std'])

In [ ]:
df.groupby(['_response'])['rt'].agg(['size','mean','std'])

In [ ]:
df.groupby(['stimulus','response'])['rt'].agg(['size','mean','std'])

In [ ]:
df.groupby(['coherence','response'])['rt'].agg(['size','mean','std'])

In [ ]:
df.groupby(['attention','response'])['rt'].agg(['size','mean','std'])

In [ ]:
df.groupby(['stimulus','coherence','response'])['rt'].agg(['size','mean','std'])

In [ ]:
df.groupby(['stimulus','attention','response'])['rt'].agg(['size','mean','std'])

In [ ]:
df.groupby(['stimulus','coherence','attention','response'])['rt'].agg(['size','mean','std'])

## ANOVA

In [514]:
# ANOVA for rt
model_aovrm3way = AnovaRM(df,
                   'rt',
                   'subj_idx',
                   within=['coherence','attention','stimulus'],
                   aggregate_func='mean')
res3way=model_aovrm3way.fit()
print(res3way)
# ANOVA for response
model_aovrm3way = AnovaRM(df,
                   'response',
                   'subj_idx',
                   within=['coherence','attention','stimulus'],
                   aggregate_func='mean')
res3way=model_aovrm3way.fit()
print(res3way)

                          Anova
                             F Value Num DF  Den DF Pr > F
----------------------------------------------------------
coherence                    37.5060 1.0000 14.0000 0.0000
attention                     0.6770 1.0000 14.0000 0.4244
stimulus                      0.0131 1.0000 14.0000 0.9105
coherence:attention           2.2672 1.0000 14.0000 0.1544
coherence:stimulus            0.0252 1.0000 14.0000 0.8762
attention:stimulus           24.0236 1.0000 14.0000 0.0002
coherence:attention:stimulus 21.9507 1.0000 14.0000 0.0004

                          Anova
                             F Value Num DF  Den DF Pr > F
----------------------------------------------------------
coherence                    14.5044 1.0000 14.0000 0.0019
attention                     0.0714 1.0000 14.0000 0.7933
stimulus                      4.3023 1.0000 14.0000 0.0570
coherence:attention          10.3949 1.0000 14.0000 0.0061
coherence:stimulus            0.5806 1.0000 14.000

## fit data